In [ ]:
import pandas as pd
import pandas_datareader.data as web
import datetime as dt
import pickle
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import timeit
style.use('ggplot')

In [ ]:
end = dt.datetime(2018,1,1)
start = dt.datetime(2016,1,1)

In [ ]:
with open('nifty50.pickle','rb') as f:  #TICKERS OF TOP 50 COMPANIES IN NSE
    tickers=pickle.load(f)

In [ ]:
if not os.path.exists('DEMA/stock_df2'):
    os.makedirs('DEMA/stock_df2')
    
for ticker in tickers:
     if not os.path.exists('DEMA/stock_df2/{}.csv'.format(ticker)):
        if ticker== 'INFRATEL.NS':
            continue
        else:
            df= web.DataReader(ticker, 'yahoo', start , end)
            df['return'] = (df['Adj Close'] -df['Adj Close'].shift(1, fill_value= df['Adj Close'][0]))/df['Adj Close'].shift(1, fill_value= df['Adj Close'][0]) *100
            df.to_csv('DEMA/stock_df2/{}.csv'.format(ticker)) 
#         df.rename(columns= {'Adj Close': ticker},inplace=True)
#         df.drop(['Open' ,'High', 'Low','Close', 'Volume'],1,inplace=True)

In [ ]:
buy_sell_hold= []
d=0
f=50   #fast MA
s=95   #slow MA
t=60   #hold period
buy_sell_hold= pd.DataFrame()
for ticker in tickers:
    if ticker !='INFRATEL.NS' and ticker !='HDFCLIFE.NS':
        df= pd.read_csv('DEMA/stock_df2/{}.csv'.format(ticker))
        df.index= df['Date']

        df['EMA_F']= df['EMA_S']= df['DEMA_S']= df['DEMA_F']= df['signal']= df['DEMA_{}'.format(ticker)]= 0
        df['EMA_F'][f] = df['Adj Close'][:f].mean()
        df['EMA_S'][s] = df['Adj Close'][:s].mean()

        df['DEMA_F'][f] = df['Adj Close'][:f].mean()        
        df['DEMA_S'][s] = df['Adj Close'][:s].mean()

        for i in range(f,len(df)):
            df['EMA_F'][i]= 2/(f+2)*df['Adj Close'][i]+ df['EMA_F'][i-1]*f/(f+2)
            df['DEMA_F'][i]= 2*df['EMA_F'][i] - 2/(f+2)*df['EMA_F'][i]- df['DEMA_F'][i-1]*f/(f+2)            

            if i> s :
                df['EMA_S'][i]= 2/(s+2)*df['Adj Close'][i]+ df['EMA_S'][i-1]*s/(s+2)
                df['DEMA_S'][i]= 2*df['EMA_S'][i] - 2/(s+2)*df['EMA_S'][i]- df['DEMA_S'][i-1]*s/(s+2)

            df['signal'][i]= (df['DEMA_F'][i]- df['DEMA_S'][i])>0
            df['DEMA_{}'.format(ticker)][i] = df['signal'][i] - df['signal'][i-1]
#             df['DEMA_{}'.format(ticker)] = df['signal'] - df['signal'].shift(1, fill_value=0)
            if df['DEMA_{}'.format(ticker)][i]!=0 and i+t<len(df):
                c=(df['Adj Close'][i]-df['Adj Close'][i+t])/df['Adj Close'][i]
                d=d+ c*df['DEMA_{}'.format(ticker)][i]
        df.rename(columns= {'Adj Close': ticker},inplace=True)        
        buy_sell_hold= buy_sell_hold.join(df['{}'.format(ticker)][s+1:] , how='outer')
        buy_sell_hold= buy_sell_hold.join(df['DEMA_{}'.format(ticker)][s+1:] , how='outer')


In [ ]:
d

In [ ]:
buy_sell_hold

In [ ]:
# for i in range(100):
#     if not os.path.exists('DEMA/buy_sell_hold{}.csv'.format(i)):
#         buy_sell_hold.to_csv('DEMA/buy_sell_hold{}.csv'.format(i))
#         break

In [ ]:
plt.figure(figsize=(25,25))
plt.plot(df['ZEEL.NS'][s:])
plt.plot(df['DEMA_F'][s:])
plt.scatter('2017-02-01',df['ZEEL.NS']['2017-02-01'])
plt.scatter('2017-01-01',df['ZEEL.NS']['2017-01-01'])
plt.plot(df['EMA_S'][s:]) 